# <center>[프로젝트] LLM 서비스 개발 과정 실습 </center>

----

# 환경 설정

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import re
import os, json
from glob import glob

from textwrap import dedent
from pprint import pprint

import uuid

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


In [4]:
from langfuse import get_client

# Langfuse 클라이언트 초기화
langfuse = get_client()

# 연결 테스트
assert langfuse.auth_check()

In [5]:
SEARCH_TOP_K = 10
DB_VERSION = 2

# 워크플로우 설계



```mermaid
flowchart TD
    A[데이터 로딩] --> B[이벤트/대화록 VectorDB 저장]
    B --> C[질문 RAG & Sub-graph 구현]
    C --> D[라우팅: 질문 분석]
    
    D --> E{질문 유형 분기}
    E -->|프레임 이미지 장면 검색| F[Video Search]
    E -->|비디오 설명 관련 검색| G[Event Search]
    E -->|대화 관련 검색| H[Dialogues Search]
    
    F --> I[완료]
    G --> I[완료]
    H --> I[완료]

    
    style A fill:#e1f5fe
    style B fill:#f3e5f5
    style C fill:#e8f5e8
    style D fill:#fff3e0
    style E fill:#fce4ec

```
## 해야할 일
* 기능&툴
    - langfuse로 프롬프트 형상 관리 & 로그 모니터링
    - Corective RAG 사용
    - gradio/streamlit Web UI 구현

* rounting
    - 비디오 속 한 장면에 대한 묘사로 질문한다면 'semantic search'
    - 비디오 전체 상황에 대한 설명으로 질문한다면 'event synopsis'
    - 비디오 속 대화 내용에 대한 질문이라면 'dialogues'

* 예상 질문:
    - '분홍색 자켓을 입은 여자가 나오는 비디오 찾아서, 어떤 상황에 처해 있는지 알려줘'
    - '경찰이 단속하는 내용이 있는 비디오 찾아서 어떤 사건들이 있었는지 시간 순으로 정리해줘'
    - 'NVIDIA 관련 발표를 하는 비디오 찾아서 주요 내용을 요약해줘' 
    - 'GPU에 대해 말하고 있는 비디오 찾아서 주요 내용을 요약해줘' 
    - '요리 방법에 대해 설명하고 있는 비디오 찾아서 그 방법을 설명해줘' 
    - '목공에 대해 설명하고 있는 비디오 찾아서 어떤 공구를 사용하는지 알려줘' 



# 데이터 준비

In [6]:
# synopsis file 목록
synopsis_files = glob(os.path.join('data', 'synopsis_*.txt'))

synopsis_files

['data/synopsis_dialogues.txt', 'data/synopsis_events.txt']

In [7]:
# 데이터 로딩
from langchain_community.document_loaders import TextLoader

## synopsis_events.txt 파일 로딩
synopsis_events_loader = TextLoader('data/synopsis_events.txt', encoding='utf-8')
synopsis_events_pages = synopsis_events_loader.load()
len(synopsis_events_pages)

## synopsis_dialogues.txt 파일 로딩
synopsis_dialogues_loader = TextLoader('data/synopsis_dialogues.txt', encoding='utf-8')
synopsis_dialogues_pages = synopsis_dialogues_loader.load()
len(synopsis_dialogues_pages)

print(f"len of synopsis_events_pages: {len(synopsis_events_pages)}")
print(synopsis_events_pages[0].page_content[:500])
print("-" * 100)
print(f"len of synopsis_dialogues_pages: {len(synopsis_dialogues_pages)}")
print(synopsis_dialogues_pages[0].page_content[:500])


len of synopsis_events_pages: 1
=== Event 1 ===
#: 1
event_id: 1
video_group_id: 1
is_important: true
time_tag: {"일반 시간"}
latitude: 37.401895
longitude: 127.102518
location: 경기 성남시 분당구 판교로 264 sk플래닛판교사옥
place_tag: {"주거 아파트"}
visual_synopsis_path: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/VID_20250602_141834_430/events/VID_20250602_141834_430_event_1.mp4
title: 가정 갈등 해결 및 분리 권고(수정본)
event_summary: 이 사건은, 서울 은평구에서 가정 갈등이 보고되었습니다. 부부가 재정 문제로 다투는 상황에서 경찰이 도착하여 임시 분리를 권고했습니다. 과거 언어적 및 정서적 폭력 사례를 언급
----------------------------------------------------------------------------------------------------
len of synopsis_dialogues_pages: 1
ID: 1 | Video: 39 | Speaker: 89 | Time: {6.14,14.902} | Text:  네 현장으로 이동 중입니다.
ID: 2 | Video: 39 | Speaker: 89 | Time: {15.022,16.703} | Text: 도착까지 약 3분 소요.
ID: 3 | Video: 39 | Speaker: 89 | Time: {16.723,22.104} | Text: 자 왔습니다.
ID: 4 | Video: 39 | Speaker: 89 | Time: {22.724,24.325} | Text: 계십니까 경찰입니다.
ID: 5 | Video: 39 | Speake

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter


event_text_splitter = CharacterTextSplitter(
    separator = "\n\n",        # 청크 구분자: 개행문자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)
dialogue_text_splitter = CharacterTextSplitter(
    separator = "\n",        # 청크 구분자: 개행문자
    chunk_size=500,         # 청크 크기
    chunk_overlap=100,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

synopsis_events_docs = event_text_splitter.split_documents(synopsis_events_pages)
print("synopsis_events_docs 청크 수:", len(synopsis_events_docs))
synopsis_dialogues_docs = dialogue_text_splitter.split_documents(synopsis_dialogues_pages)
print("synopsis_dialogues_docs 청크 수:", len(synopsis_dialogues_docs))


Created a chunk of size 519, which is longer than the specified 500
Created a chunk of size 518, which is longer than the specified 500
Created a chunk of size 518, which is longer than the specified 500
Created a chunk of size 798, which is longer than the specified 500
Created a chunk of size 519, which is longer than the specified 500
Created a chunk of size 520, which is longer than the specified 500


synopsis_events_docs 청크 수: 126
synopsis_dialogues_docs 청크 수: 2406


# 랭체인 Document 객체에 메타데이터와 함께 정리

- 벡터저장소에 인덱싱

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI 임베딩 모델 생성
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")

# 한국어 문서를 저장하는 벡터 저장소 생성
db_synopsis_events = Chroma.from_documents(
    documents=synopsis_events_docs, 
    embedding=embeddings_openai,
    collection_name=f"synopsis_events_docs_{DB_VERSION}",
    persist_directory="./chroma_db",
)
print(f"db_synopsis_events 문서 수: {db_synopsis_events._collection.count()}")

db_synopsis_dialogues = Chroma.from_documents(
    documents=synopsis_dialogues_docs, 
    embedding=embeddings_openai,
    collection_name=f"synopsis_dialogues_docs_{DB_VERSION}",
    persist_directory="./chroma_db",
)
print(f"db_synopsis_dialogues 문서 수: {db_synopsis_dialogues._collection.count()}")




db_synopsis_events 문서 수: 378
db_synopsis_dialogues 문서 수: 7218


# 도구 호출

### synopsis_event, synopsis_dialogues 검색 도구, video(B/E APIs) 검색 도구 정의

- video 검색(semantic search)는 해당 vector db에 바로 접근할 수 없어서 metavision api 사용 


In [164]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_core.tools import tool
from typing import List

from urllib.parse import quote
import requests

# 문서 임베딩 모델
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Re-rank 모델
rerank_model = HuggingFaceCrossEncoder(
    model_name="Alibaba-NLP/gte-multilingual-reranker-base",
    model_kwargs={
        "device": "cpu",  # CPU에서 실행
        "trust_remote_code": True,  # 모델이 외부 코드를 신뢰하도록 설정
        } 
    )
cross_reranker = CrossEncoderReranker(model=rerank_model, top_n=SEARCH_TOP_K)


db_synopsis_events_retriever = ContextualCompressionRetriever(
    base_compressor=cross_reranker, 
    base_retriever=db_synopsis_events.as_retriever(search_kwargs={"k":SEARCH_TOP_K}),
)

db_synopsis_dialogues_retriever = ContextualCompressionRetriever(
    base_compressor=cross_reranker, 
    base_retriever=db_synopsis_dialogues.as_retriever(search_kwargs={"k":SEARCH_TOP_K}),
)

@tool
def synopsis_events_search(query: str) -> List[Document]:
    """저장된 비디오의 요약 및 내용에 대해서 검색하고 관련된 내용 및 재생 링크를 제공공니다.
    결과 형식:
    - title: [title 값]
    - 내용: [event_summary 값]
    - 태그: [event_hashtag 값]
    - video_link: [visual_synopsis_path 값] 예, 'http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/GX018674/events/GX018674_event_89.mp4'
    
    """
    docs = db_synopsis_events_retriever.invoke(query)

    if len(docs) > 0:
        return docs

    return [Document(page_content="**SynapsEgo에서 검색**에서 관련 정보를 찾을 수 없습니다.", metadata={"source": "data/synopsis_events.txt", "name": "SynapsEgo"})]


@tool
def synopsis_dialogues_search(query: str) -> List[Document]:
    """저장된 비디오의 대화 내용에 대해서 검색하고 관련된 내용 및 video_id와 시간 정보를 제공공니다.
    결과 형식:
    - video_id: [Video 값]
    - 시간: [Time 값], 예, {22.724,24.325} 의 경우 22.724 ~ 24.325 사이의 시간 범위를 의미
    - 대화내용: [Text 값]
     """
    # tool_calls를 사용했는지 확인하려면
    # tool_calls가 있다면 tool_calls를 사용하고, 없으면 기존 방식대로 동작
    if hasattr(db_synopsis_dialogues_retriever, "tool_calls"):
        docs = db_synopsis_dialogues_retriever.tool_calls(query)
        print(f"db_synopsis_dialogues_retriever.tool_calls 호출 결과: {docs}")
    else:
        docs = db_synopsis_dialogues_retriever.invoke(query)
        print(f"db_synopsis_dialogues_retriever.invoke 호출 결과: {docs}")

    if len(docs) > 0:
        return docs

    return [Document(page_content="**SynapsEgo에서 검색**에서 관련 정보를 찾을 수 없습니다.", metadata={"source": "data/synopsis_dialogues.txt", "name": "SynapsEgo"})]


# 웹 검색
web_retriever = ContextualCompressionRetriever(
    base_compressor=cross_reranker, 
    base_retriever=TavilySearchAPIRetriever(k=5),
)

def video_search(query: str= None) -> str:
    page_size = 20
    page_num = 1
    input_prompt = ""

    #print(f"synopsis search url : {url}")
    preview_url_prefix = "http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/"

    while True:
        url = "http://metavision.k8s.lightningdb/api/metavision-ego-view-ai/api/v1/video/search"
        url = url +f"?sort_method=newest_created_first&page_size={page_size}&page_num={page_num}"
        url = url + f"&search_keyword={quote(query)}" if query else url
        headers = {"Content-Type": "application/json"}  # 요청 헤더: JSON 형식의 데이터를 전송한다는 의미
    
        response = requests.get(url,headers=headers)
        response.raise_for_status()  # HTTP 에러가 발생했을 경우 예외를 발생시킴

        # 응답 본문을 JSON으로 변환 후 "results" 항목 추출 (리스트 형태)
        results = response.json().get("rowset", [])
        #print(f"Fetched {results} results from page {page_num}.")

        # 결과가 없을 경우 사용자에게 안내
        if not results:
            break

        # 각 결과 항목의 제목과 내용을 합쳐 하나의 문자열로 구성 (결과 여러 개를 줄 구분으로 연결)
        for r in results:
            for video in r.get('video_list', []):
                video_name = video.get('video_name', "N/A")
                video_link_raw = video.get('video_link', "N/A")
                video_link = f"[영상 보기]({video_link_raw})"
                video_thumbnail = video.get('video_thumbnail', "N/A")
                created_time = video.get('created_time', "N/A")
                contents = f"- 파일명: {video_name}, - 비디오 링크: {video_link}, - 썸네일: {video_thumbnail}, - 생성시간: {created_time}"
                input_prompt += contents + "\n\n"

        current_items = len(results)
        if current_items < page_size:
            print(f"last page reached: {page_num}")
            break
        page_num += 1

    return str(input_prompt)[:10000]  # 완성된 검색 결과 문자열 반환

@tool
def metavision_search(query: str) -> List[str]:
    """저장된 비디오의 주요 장면에 대한 설명으로 비디오를 검색합니다.
    예,
         "video_list": [
        {
          "video_id": 91,
          "video_name": "POV： Cooking Restaurant Quality Chicken (How To Make it at Home)_h264.mp4",
          "device": "bodycam-0",
          "created_time": "2025-08-26 10:09:24",
          "video_link": "http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/POV： Cooking Restaurant Quality Chicken (How To Make it at Home)_h264.mp4",
          "video_thumbnail": "http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/POV： Cooking Restaurant Quality Chicken (How To Make it at Home)_h264.mp4/grid"
        }, ...
    """
    
    # TODO

    docs = video_search(query)

    formatted_docs = docs


    if len(formatted_docs) > 0:
        return formatted_docs

    return [Document(page_content="**웹 검색**에서 관련 정보를 찾을 수 없습니다.")]

In [165]:
@tool
def get_video_link(video: int) -> str:
    """ video를 입력받아 비디오 링크를 반환합니다. """
    url_prefix = "http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3"
    
    request_url=f"http://metavision.k8s.lightningdb/api/metavision-ego-view-ai/api/v1/video/name?video_id={video}"
    response = requests.get(request_url)
    response.raise_for_status()
    output = f"{url_prefix}/original_videos/{response.text}"
    print(f"converted link: {output}")
    return output


In [166]:
# 도구 목록을 정의 
tools = [synopsis_events_search, synopsis_dialogues_search, metavision_search, get_video_link]


### 3-2. LLM 모델

In [167]:
from langchain_openai import ChatOpenAI

# 기본 LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# LLM에 도구 바인딩하여 추가 
llm_with_tools = llm.bind_tools(tools)

In [168]:
# 장면에 대한 묘사로 검색 도구를 호출  
query = "분홍색 자켓을 입은 여자가 나오는 비디오 찾아서, 어떤 상황에 처해 있는지 알려줘."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_utPUHBYtRyGMK5c1l7nZnYaT', 'function': {'arguments': '{"query":"분홍색 자켓을 입은 여자"}', 'name': 'synopsis_events_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 510, 'total_tokens': 534, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CLpVBDarh2KTMuWHNmFci5412lknm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--da1be837-0387-4bb0-abd3-99450c7674c6-0', tool_calls=[{'name': 'synopsis_events_search', 'args': {'query': '분홍색 자켓을 입은 여자'}, 'id': 'call_utPUHBYtRyGMK5c1l7nZnYaT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 510, 'output_tokens': 24, 'total_

In [15]:
# 비디오 속 상황에 대한 묘사로 검색 도구를 호출  
query = "경찰이 단속을 하는 내용이 있는 비디오 찾아줘"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qM9kE85DabBMlAhKNcdz1MeD', 'function': {'arguments': '{"query":"경찰 단속"}', 'name': 'synopsis_events_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 497, 'total_tokens': 516, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CLlFKZpKeXO9tosvOvCQmvy7yyjQJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9a04dfa6-e500-4f73-850b-e6d4e4cb0992-0', tool_calls=[{'name': 'synopsis_events_search', 'args': {'query': '경찰 단속'}, 'id': 'call_qM9kE85DabBMlAhKNcdz1MeD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 497, 'output_tokens': 19, 'total_tokens': 516, 'i

In [16]:
# 벡터 검색과 웹 검색이 모두 필요한 경우 
query = "경찰이 단속하는 내용이 있는 비디오에서 음주 운전 관련한 대화가 있는 비디오 찾아줘"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_S5wbcFq6h60ThXXxOCCUMwuM', 'function': {'arguments': '{"query":"음주 운전"}', 'name': 'synopsis_dialogues_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 509, 'total_tokens': 529, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CLlFNyI0HWWt14GQPVz0MGkTlIopZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ac8f1cb9-e44c-4b62-bc62-b6aa0a71816d-0', tool_calls=[{'name': 'synopsis_dialogues_search', 'args': {'query': '음주 운전'}, 'id': 'call_S5wbcFq6h60ThXXxOCCUMwuM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 509, 'output_tokens': 20, 'total_tokens': 5

In [17]:
# 비디오 링크 가져오기
query = "video 146에 대한 비디오 링크 찾아줘"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ia9XcwVjdbYekbVH0yMBecCa', 'function': {'arguments': '{"video":146}', 'name': 'get_video_link'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 493, 'total_tokens': 508, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CLlFSfOfGkJyAeZKJgBqAIiAmgBQM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b968c707-4b53-4fa4-a16c-e87a658e6630-0', tool_calls=[{'name': 'get_video_link', 'args': {'video': 146}, 'id': 'call_Ia9XcwVjdbYekbVH0yMBecCa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 493, 'output_tokens': 15, 'total_tokens': 508, 'input_token_details': {'a

# Agent RAG 구현 

- 각 질문에 특화된 RAG 에이전트를 구현 
- 질문 라우팅을 통해서 각 에이전트를 도구 형태로 사용 
- 생성된 답변에 대한 피드백을 제공하는 에이전트 사용 
- 필요한 경우 사람의 피드백을 요청 (답변이 애매한 경우 - 재검색 여부 판단)


### 각 질문에 특화된 RAG 에이전트를 구현 
- 검색된 문서의 관련성 등을 평가하여 질문 재작성 및 다시 검색 (Corrective RAG 적용)

`(1) Synopsis event 검색 에이전트`

In [18]:
from pydantic import BaseModel, Field
from typing import List, TypedDict, Optional, Annotated, Literal
from langchain_core.documents import Document
import operator

class CorrectiveRagState(TypedDict):
    question: str                 # 사용자의 질문
    generation: str               # LLM 생성 답변
    documents: List[Document]     # 컨텍스트 문서 (검색된 문서)
    num_generations: int          # 질문 or 답변 생성 횟수 (무한 루프 방지에 활용)
    user_decision: Literal["rejected", "accepted"]  # 사용자 결정 (거부 or 수락)
    user_feedback: Optional[str]  # 사용자 피드백 (구체적인 피드백을 제공할 수 있는 경우)

class InformationStrip(BaseModel):
    """추출된 정보에 대한 내용과 출처, 관련성 점수"""
    content: str = Field(description="추출된 정보 내용")
    video_link: str = Field(description="추출된 정보 영상 링크")
    tags: str = Field(description="추출된 정보 태그")
    title: str = Field(description="추출된 정보 제목")
    location: str = Field(description="추출된 정보 위치")
    start_time: str = Field(description="추출된 정보 시작 시간")
    end_time: str = Field(description="추출된 정보 종료 시간")
    source: str = Field(description="정보의 출처")
    relevance_score: float = Field(description="관련성 점수 (0에서 1 사이)")
    faithfulness_score: float = Field(description="충실성 점수 (0에서 1 사이)")

class ExtractedInformation(BaseModel):
    strips: List[InformationStrip] = Field(description="추출된 정보 조각들")
    query_relevance: float = Field(description="질의에 대한 전반전인 답변 가능성 점수 (0에서 1 사이)")

class RefinedQuestion(BaseModel):
    """개선된 질문과 이유"""
    question_refined : str = Field(description="개선된 질문")
    reason : str = Field(description="이유")

# SynopsisEventRagState - 병렬 처리를 지원
class SynopsisEventRagState(CorrectiveRagState):
    rewritten_query: str   # 재작성한 질문 
    extracted_info: Annotated[List[InformationStrip], operator.add]  # 병렬 워커 작업용
    processed_info: List[InformationStrip]  # 후처리된 결과용 
    node_answer: Optional[str]

# 개별 문서 처리를 위한 워커 상태
class DocumentWorkerState(TypedDict):
    document: str  # 단일 문서
    question: str  # 질문
    # 워커에서도 동일한 타입으로 반환
    extracted_info: Annotated[List[InformationStrip], operator.add]

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langgraph.types import Send

def retrieve_documents(state: SynopsisEventRagState) -> SynopsisEventRagState:
    print("---Synopsis event 검색---")
    query = state.get("rewritten_query", state["question"])
    docs = synopsis_events_search.invoke(query)
    return {"documents": docs}

def create_evaluation_workers(state: SynopsisEventRagState):
    """각 문서에 대해 평가 작업을 수행하는 병렬 워커를 생성하는 조건부 엣지 함수"""
    print(f"---Synopsis event 관련 {len(state['documents'])}개 문서에 대한 평가 병렬 워커 생성---")

    # 각 문서에 대해 Send 객체를 생성하여 병렬 처리
    return [
        Send("evaluate_single_document", {
            "document": doc.page_content,
            "question": state["question"],
            "user_feedback": state.get("user_feedback", ""),  # 사용자 피드백이 있는 경우 전달
        })
        for doc in state["documents"]
    ]

def evaluate_single_document(state: DocumentWorkerState) -> SynopsisEventRagState:
    """단일 문서를 처리하는 워커 함수"""
    print(f"---Synopsis event 단일 문서 처리 중 (길이: {len(state['document'])}...)---")
    
    
    extract_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 비디오 분석 전문가입니다. 주어진 문서에서 질문과 관련된 주요 사실과 정보를 최대 10개 정도 추출하세요. 
석
        각 추출된 정보에 대해 다음 두 가지 측면을 0에서 1 사이의 점수로 평가하세요:
        1. 질문과의 관련성
        2. 답변의 충실성 (질문에 대한 완전하고 정확한 답변을 제공할 수 있는 정도)
        3. 문서에서 event 단위로 평가 진행
        
        추출 형식:
        1. [추출된 정보]
        - 이벤트별로 제공된 정보는 다음과 같음
            - 제목: title 값
            - 내용: event_summary 값
            - 태그: event_hashtag 값
            - 영상 링크: video_link 값
            - 위치: location 값
            - 시작 시간: start_timestamp 값
            - 종료 시간: end_timestamp 값
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        2. [추출된 정보]
        - 이벤트별로 제공된 정보는 다음과 같음
            - 제목: title 값
            - 내용: event_summary 값
            - 태그: event_hashtag 값
            - 영상 링크: video_link 값
            - 위치: location 값
            - 시작 시간: start_timestamp 값
            - 종료 시간: end_timestamp 값
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        ...
        
        마지막으로, 추출된 정보를 종합하여 질문에 대한 전반적인 답변 가능성을 0에서 1 사이의 점수로 평가하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n<문서 내용>\n{document_content}\n</문서 내용>")
    ])
    

    # LLM 호출 
    extract_llm = llm_with_tools.with_structured_output(ExtractedInformation)
    
    try:
        extracted_data = extract_llm.invoke(extract_prompt.format(
            question=state["question"],
            document_content=state["document"],
            user_feedback=[("system", f"정보를 추출할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state['user_feedback']}</피드백>")] if "user_feedback" in state else []
        ))

        # 품질 필터링
        if extracted_data.query_relevance < 0.8:
            print(f"문서 관련성이 낮음: {extracted_data.query_relevance}")
            return {"extracted_info": []}

        # 고품질 정보만 필터링
        high_quality_strips = [
            strip for strip in extracted_data.strips
            if strip.relevance_score > 0.7 and strip.faithfulness_score > 0.7
        ]
        
        print(f"고품질 정보 {len(high_quality_strips)}개 추출됨")
        return {"extracted_info": high_quality_strips}
        
    except Exception as e:
        print(f"문서 처리 중 오류 발생: {e}")
        return {"extracted_info": []}

def aggregate_results(state: SynopsisEventRagState) -> SynopsisEventRagState:
    """병렬 처리된 결과를 집계하는 함수"""
    print(f"---Synopsis event 관련 총 {len(state['extracted_info'])}개의 정보 조각 집계---")

    # 후처리가 필요한 경우에만 작업 추가 
    
    # 방법 1: 중복 제거
    unique_strips = []
    seen_content = set()
    for strip in state["extracted_info"]:
        if strip.content not in seen_content:
            unique_strips.append(strip)
            seen_content.add(strip.content)
    
    # 방법 2: 품질순 정렬
    sorted_strips = sorted(
        unique_strips, 
        key=lambda x: (x.relevance_score + x.faithfulness_score) / 2, 
        reverse=True
    )
    
    # 방법 3: 상위 N개만 선택
    top_strips = sorted_strips[:SEARCH_TOP_K]  # 상위 10개만
    
    return {
        "num_generations": state.get("num_generations", 0) + 1,
        "processed_info": top_strips  # 후처리된 결과
    }

In [20]:
# 쿼리 재작성 함수
def rewrite_query(state: SynopsisEventRagState) -> SynopsisEventRagState:
    print("---Synopsis event 쿼리 재작성---")

    rewrite_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 비디오 분석 전문가입니다. 
         주어진 원래 질문과 추출된 정보를 바탕으로, 더 관련성 있고 충실한 정보를 찾기 위해 검색 쿼리를 개선해주세요.
         검색된 내용에 실행 가능한 링크가 있으면 반드시 추가해주세요.

        다음 사항을 고려하여 검색 쿼리를 개선하세요:
        1. 원래 질문의 핵심 요소
        2. 추출된 정보의 관련성 점수
        3. 추출된 정보의 충실성 점수
        4. 부족한 정보나 더 자세히 알아야 할 부분

        개선된 검색 쿼리 작성 단계:
        1. 2-3개의 검색 쿼리를 제안하세요.
        2. 각 쿼리는 구체적이고 간결해야 합니다(5-10 단어 사이).
        3. 각 쿼리 뒤에는 해당 쿼리를 제안한 이유를 간단히 설명하세요.

        출력 형식:
        1. [개선된 검색 쿼리 1]
        - 이유: [이 쿼리를 제안한 이유 설명]
        2. [개선된 검색 쿼리 2석
        - 이유: [이 쿼리를 제안한 이유 설명]
        3. [개선된 검색 쿼리 3]
        - 이유: [이 쿼리를 제안한 이유 설명]

        마지막으로, 제안된 쿼리 중 가장 효과적일 것 같은 쿼리를 선택하고 그 이유를 설명하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<원래 질문>\n{question}\n</원래 질문>\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 개선된 검색 쿼리를 작성해주세요.")
    ])

    processed_info_str = "\n".join([strip.content for strip in state["processed_info"]])
    
    rewrite_llm = llm_with_tools.with_structured_output(RefinedQuestion)

    response = rewrite_llm.invoke(rewrite_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"쿼리를 재작성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))
    
    return {"rewritten_query": response.question_refined}

def generate_node_answer(state: SynopsisEventRagState) -> SynopsisEventRagState:
    print("---Synopsis event 답변 생성---")

    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 비디오 분석 전문가입니다. 주어진 질문과 추출된 정보를 바탕으로 답변을 생성해주세요. 
        답변은 마크다운 형식으로 작성하며, 각 정보의 출처를 명확히 표시해야 합니다. 
        답변 구조:
        1. 검색된 결과는 이벤트 별로 그대로 제공
        예,
        - 제목: '경찰 단속 사건'
         - 내용: '경찰복을 입은 경찰관이 신체 카메라를 사용하여 도시 거리에서 흰색 트래버스 SUV(번호판 236조 2693)를 정지시켜 교통 위반 혐의를 통보함.', 
         - 영상 링크: 'http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/GX018665/events/GX018665_event_90.mp4', 
         - 태그: '#교통위반,#빨간불카메라,#운전벌점'
         - 장소: '서울시 강남구 테헤란로 14길'
         - 시간: '2025-09-25 10:00:00 ~ 2025-09-25 10:01:00'
         - 출처: 원본 파일이름
        2. 결론 및 요약
         - 몇 개의 이벤트가 검출되었는지와 검출된 이벤트의 모든 태그 정보를 중복을 제거하고 정리해서 알려줘. 태그는 중복이 있으면 뒤에 그 숫자를 '(3)'와 같이 붙여줘
        각 섹션에서 사용된 정보의 출처를 괄호 안에 명시하세요. """),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 최종 답변을 작성해주세요.")
    ])

    processed_info_str = "\n---\n".join([f"**제목**: {strip.title}\n **내용**: {strip.content}\n **영상 링크**: {strip.video_link}\n **태그**: {strip.tags}\n **시간**: {strip.start_time} ~ {strip.end_time}\n **출처**: {strip.source}\n **관련성**: {strip.relevance_score}\n **충실성**: {strip.faithfulness_score}" for strip in state["processed_info"]])

    node_answer = llm.invoke(answer_prompt.format(
        question=state["question"],
            processed_info=processed_info_str,
        user_feedback=[("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))

    return {"node_answer": node_answer.content}

def should_continue(state: SynopsisEventRagState) -> Literal["계속", "종료"]:
    if state["num_generations"] >= 2:
        return "종료"
    if len(state["processed_info"]) > 0:
        return "종료"
    return "계속"

In [21]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, Markdown, display


# 그래프 생성
workflow = StateGraph(SynopsisEventRagState)

workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("evaluate_single_document", evaluate_single_document)
workflow.add_node("aggregate_results", aggregate_results)
workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("generate_answer", generate_node_answer)

workflow.add_edge(START, "retrieve")

# 조건부 엣지로 병렬 처리 설정
workflow.add_conditional_edges(
    "retrieve",  # 문서 검색 완료 후
    create_evaluation_workers,  # 조건부 엣지 함수
    ["evaluate_single_document"]  # 병렬 워커 노드
)

workflow.add_edge("evaluate_single_document", "aggregate_results")

# 조건부 엣지 추가
workflow.add_conditional_edges(
    "aggregate_results",
    should_continue,
    {
        "계속": "rewrite_query",
        "종료": "generate_answer"
    }
)
workflow.add_edge("rewrite_query", "retrieve")
workflow.add_edge("generate_answer", END)

# 그래프 컴파일
synopsis_event_agent = workflow.compile()

# 그래프 시각화
# display(Image(personal_law_agent.get_graph().draw_mermaid_png()))
mermaid_code = synopsis_event_agent.get_graph(xray=True).draw_mermaid()
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retrieve(retrieve)
	evaluate_single_document(evaluate_single_document)
	aggregate_results(aggregate_results)
	rewrite_query(rewrite_query)
	generate_answer(generate_answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> retrieve;
	aggregate_results -. &nbsp;종료&nbsp; .-> generate_answer;
	aggregate_results -. &nbsp;계속&nbsp; .-> rewrite_query;
	evaluate_single_document --> aggregate_results;
	retrieve -.-> evaluate_single_document;
	rewrite_query --> retrieve;
	generate_answer --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [22]:
inputs = {"question": "경찰 단속과 관련된 내용이 있는 비디오를 찾아줘."}
step_no = 0
for output in synopsis_event_agent.stream(inputs):
    step_no+=1
    for key, value in output.items():
        # 노드 출력
        pprint(f"================ Step {step_no} ==================")
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)

    print("\n----------------------------------------------------------\n")

---Synopsis event 검색---
---Synopsis event 관련 10개 문서에 대한 평가 병렬 워커 생성---
'================ Step 1 =================='
"Node 'retrieve':"
("Value: {'documents': [Document(id='f6ef4e77-ed40-4db5-a2a4-50e4795ab915', "
 "metadata={'source': 'data/synopsis_events.txt'}, page_content='=== Event 4 "
 '===\\n#: 4\\nevent_id: 5\\nvideo_group_id: 2\\nis_important: '
 'true\\ntime_tag: {오후}\\nlatitude: 37.401895\\nlongitude: '
 '127.102518\\nlocation: 경기 성남시 분당구 판교로 264 sk플래닛판교사옥\\nplace_tag: {"도심 '
 '거리"}\\nvisual_synopsis_path: '
 'http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/VID_20250530_105623_421/events/VID_20250530_105623_421_event_5.mp4\\ntitle: '
 '경찰관에 의한 화이트 SUV 교통단속\\nevent_summary: 영상은 도심 거리에서 정장 차림의 경찰관과 화이트 SUV 간의 '
 '교통단속을 보여줍니다. 경찰관은 차량에 접근하여 운전자와 대화를 나누고, 문서를 확인하며 클립보드를 사용합니다. 안경을 쓴 운전자는 '
 '안전벨트를 조정하고 손짓을 하며 경찰관과 상호작용합니다. 일부 클립에는 승객이 존재하며, 장면을 관찰하거나 노트북을 보고 있습니다. '
 '장면은 나무가 심어진 도심 환경에서 발생하며, 맑은 하늘 아래 있습니다. 첫 번째 영상에는 오디오가 없으며, 시각적 서사만 '
 '제공됩\\nevent

In [23]:
# 마크다운 형식을 노트북에 표시
from IPython.display import Markdown
display(Markdown(value['node_answer']))

```markdown
## 검색된 결과

- 제목: 경찰관에 의한 화이트 SUV 교통단속  
  - 내용: 영상은 도심 거리에서 정장 차림의 경찰관과 화이트 SUV 간의 교통단속을 보여줍니다. 경찰관은 차량에 접근하여 운전자와 대화를 나누고, 문서를 확인하며 클립보드를 사용합니다. 안경을 쓴 운전자는 안전벨트를 조정하고 손짓을 하며 경찰관과 상호작용합니다. 일부 클립에는 승객이 존재하며, 장면을 관찰하거나 노트북을 보고 있습니다. 장면은 나무가 심어진 도심 환경에서 발생하며, 맑은 하늘 아래 있습니다. 첫 번째 영상에는 오디오가 없으며, 시각적 서사만 제공됩니다.  
  - 영상 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/VID_20250530_105623_421/events/VID_20250530_105623_421_event_5.mp4)  
  - 태그: #교통단속,#경찰상호작용,#도심환경  
  - 시간: 2025-07-29 16:44:49.000 ~ 2025-07-29 16:48:39.000  
  - 출처: 문서 내용

- 제목: 빨간불 위반으로 인한 교통 정지  
  - 내용: 경찰복을 입은 경찰관이 신체 카메라를 사용하여 경기 성남시 분당구 판교로 264 sk플래닛판교사옥 인근 도시 거리에서 흰색 트래버스 SUV(번호판 236조 2693)를 정지시켜 교통 위반 혐의를 통보함. 좌회전 중 빨간불 위반 사실을 교통 카메라 영상으로 확인하고 최민호 씨에게 알림. 운전자는 처음에 위반 사실을 부인했으나 경찰관이 증거를 설명함. 교통법 제5조에 따라 15점 벌점과 70,000원 벌금 부과됨.  
  - 영상 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/GX018665/events/GX018665_event_90.mp4)  
  - 태그: #교통위반,#빨간불카메라,#운전벌점  
  - 시간: 2025-08-04 20:03:05.000 ~ 2025-08-04 20:06:47.610  
  - 출처: 문서 내용

- 제목: 공원에서의 야간 경찰 체포  
  - 내용: 영상은 야간 공원에서 경찰 체포를 보여줍니다. 경찰관은 파란색 차량에 접근해 운전자(여성)와 소통하고, 뒤로 돌아서 손을 등 뒤로 놓으라고 지시합니다. 여성은 잠시 저항한 후 준수합니다. 경찰관은 여성에게 손을 등 뒤로 놓으라고 지시하고, 경찰 차량으로 안내하며 체포를 완료합니다. 이 모든 사건은 가로등 조명 아래에서 나무와 주거 지역 건물 배경에서 발생했습니다.  
  - 영상 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/Police_Arrest1/events/Police_Arrest1_event_139.mp4)  
  - 태그: #경찰체포,#야간체포,#주거지역거리  
  - 시간: 2025-08-29 14:54:16.000 ~ 2025-08-29 14:55:19.000  
  - 출처: 문서 내용

## 결론 및 요약

- 총 3개의 경찰 단속 및 관련 사건 영상이 검출되었습니다.  
- 검출된 이벤트의 태그 목록 (중복 제거 및 개수 표기):  
  - #교통단속  
  - #경찰상호작용  
  - #도심환경  
  - #교통위반  
  - #빨간불카메라  
  - #운전벌점  
  - #경찰체포  
  - #야간체포  
  - #주거지역거리  

(출처: 문서 내용)
```

`Dialogues 검색 RAG 에이전트`

In [24]:
class DialoguesStrip(BaseModel):
    """추출된 정보에 대한 내용과 출처, 관련성 점수"""
    content: str = Field(description="추출된 정보 내용. 대화 내용")
    video: int = Field(description="추출된 Video 값")
    video_link: str = Field(description="Video 값으로 구한 video link")
    time: List[float] = Field(description=f"추출된 Time 정보 시간대. 예: {55.538, 57.759}")
    source: str = Field(description="정보의 출처")
    relevance_score: float = Field(description="관련성 점수 (0에서 1 사이)")
    faithfulness_score: float = Field(description="충실성 점수 (0에서 1 사이)")

class DialoguesExtractedInformation(BaseModel):
    strips: List[DialoguesStrip] = Field(description="추출된 정보 조각들")
    query_relevance: float = Field(description="질의에 대한 전반전인 답변 가능성 점수 (0에서 1 사이)")

class DialoguesRefinedQuestion(BaseModel):
    """개선된 질문과 이유"""
    question_refined : str = Field(description="개선된 질문")
    reason : str = Field(description="이유")

# SynopsisEventRagState - 병렬 처리를 지원
class SynopsisDialogueRagState(CorrectiveRagState):
    rewritten_query: str   # 재작성한 질문 
    extracted_info: Annotated[List[DialoguesStrip], operator.add]  # 병렬 워커 작업용
    processed_info: List[DialoguesStrip]  # 후처리된 결과용 
    node_answer: Optional[str]

# 개별 문서 처리를 위한 워커 상태
class DialoguesDocumentWorkerState(TypedDict):
    document: str  # 단일 문서
    question: str  # 질문
    # 워커에서도 동일한 타입으로 반환
    extracted_info: Annotated[List[DialoguesStrip], operator.add]

In [25]:

class SynopsisDialoguesRagState(CorrectiveRagState):
    rewritten_query: str   # 재작성한 질문 
    extracted_info: Annotated[List[DialoguesStrip], operator.add]  # 병렬 워커들로부터 여러 정보 조각을 수집
    processed_info: List[DialoguesStrip]  # 후처리된 결과용 
    node_answer: Optional[str]

In [26]:
# 비디오 링크 가져오기
query = "video 146에 대한 비디오 링크 찾아줘"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9QXqaTzAsM6YZHxqaFkTK713', 'function': {'arguments': '{"video":146}', 'name': 'get_video_link'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 493, 'total_tokens': 508, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4fce0778af', 'id': 'chatcmpl-CLlHkkzEQPmu3luQcsA0q6KGdDsO0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a2ba4113-242f-4817-95ce-bd4461ab217f-0', tool_calls=[{'name': 'get_video_link', 'args': {'video': 146}, 'id': 'call_9QXqaTzAsM6YZHxqaFkTK713', 'type': 'tool_call'}], usage_metadata={'input_tokens': 493, 'output_tokens': 15, 'total_tokens': 508, 'input_token_details': {'a

In [27]:
def video_link(video: int) -> str:
    """ video를 입력받아 비디오 링크를 반환합니다. """
    url_prefix = "http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3"
    
    request_url=f"http://metavision.k8s.lightningdb/api/metavision-ego-view-ai/api/v1/video/name?video_id={video}"
    response = requests.get(request_url)
    response.raise_for_status()
    output = f"{url_prefix}/original_videos/{response.text}"
    print(f"converted link: {output}")
    return output


In [169]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal, List
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_core.documents import Document

def retrieve_documents(state: SynopsisDialoguesRagState) -> SynopsisDialoguesRagState:
    print("---synopsis dialogues 문서 검색---")
    query = state.get("rewritten_query", state["question"])
    docs = synopsis_dialogues_search.invoke(query)
    print(docs)
    return {"documents": docs}


def create_evaluation_workers(state: SynopsisDialoguesRagState):
    """각 문서에 대해 평가 작업을 수행하는 병렬 워커를 생성하는 조건부 엣지 함수"""
    print(f"---synopsis dialogues 관련 {len(state['documents'])}개 문서에 대한 평가 병렬 워커 생성---")

    # 각 문서에 대해 Send 객체를 생성하여 병렬 처리
    return [
        Send("evaluate_single_document", {
            "document": doc.page_content,
            "question": state["question"]
        })
        for doc in state["documents"]
    ]
   

def evaluate_single_document(state: DialoguesDocumentWorkerState) -> SynopsisDialoguesRagState:
    """Dialogues 단일 문서를 처리하는 워커 함수"""
    print(f"---synopsis dialogues 단일 문서 처리 중 (길이: {len(state['document'])}...)---")

    # 프롬프트 템플릿 정의
    extract_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 비디오 및 오디오 분석 전문가입니다. 주어진 문서에서 질문과 관련된 주요 사실과 정보를 10개 정도 추출하세요. 
        각 추출된 정보에 대해 다음 두 가지 측면을 0에서 1 사이의 점수로 평가하세요:
        1. 질문과의 관련성
        2. 답변의 충실성 (질문에 대한 완전하고 정확한 답변을 제공할 수 있는 정도)
        - 대화별로 제공된 정보는 다음과 같음
            - video: Video 값
            - video_link: video 값으로 비디오 링크 찾아서 추가
            - 내용: Text 값
            - 시간: 대화 시간대 정보(Time 값, '55.538 ~ 57.759' 와 같이 표시)
            - 관련성 점수: [0-1 사이의 점수]
            - 충실성 점수: [0-1 사이의 점수]

        ...  
                    
        마지막으로, 추출된 정보를 종합하여 질문에 대한 전반적인 답변 가능성을 0에서 1 사이의 점수로 평가하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n<문서 내용>\n{document_content}\n</문서 내용>")
    ])
    print("프롬프트 템플릿 정의")
    # LLM 호출
    extract_llm = llm_with_tools.with_structured_output(DialoguesExtractedInformation)  

    print(f"LLM 호출 with video: {state}")
    try:    
        print(f"질문: {state['question']}, 문서: {state['document']}, 피드백: {state.get('user_feedback', '')}")
        prompt_temp = extract_prompt.invoke( {'question': state["question"], 'document_content': state["document"], 'user_feedback': [("system", f"정보를 추출할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []})
        print(f"프롬프트: {prompt_temp}")
        extracted_data = extract_llm.invoke(prompt_temp)
        
        print(f"---> LLM 호출 완료:video:{extracted_data}")
        # ---> LLM 호출 완료:video:strips=[DialoguesStrip(content='그들은 GPUs를 운영합니다.', video=201, video_link='https://example.com/video/201', time=[766.493, 770.516], source='문서 내용', relevance_score=1.0, faithfulness_score=1.0), DialoguesStrip(content='GPU 클라우드라고 부릅니다.', video=201, video_link='https://example.com/video/201', time=[770.816, 772.557], source='문서 내용', relevance_score=1.0, faithfulness_score=1.0), DialoguesStrip(content='우리의 대단한 파트너 CoreWeave는 공공장소에서 공개되고 있습니다.', video=201, video_link='https://example.com/video/201', time=[772.577, 775.579], source='문서 내용', relevance_score=0.7, faithfulness_score=0.8), DialoguesStrip(content='정말 자랑스럽습니다.', video=201, video_link='https://example.com/video/201', time=[775.619, 776.479], source='문서 내용', relevance_score=0.3, faithfulness_score=0.3), DialoguesStrip(content='GPU 클라우드는 자기들만의 요구를 가지고 있습니다.', video=201, video_link='https://example.com/video/201', time=[776.499, 777.68], source='문서 내용', relevance_score=1.0, faithfulness_score=0.9)] query_relevance=1.0



#---> LLM 호출 완료:video:strips=[DialoguesStrip(content='그들은 GPUs를 운영합니다.', video=196, video_link='https://example.com/video/196', time=[766.493, 770.516], source='ID: 10346', relevance_score=1.0, faithfulness_score=1.0), DialoguesStrip(content='GPU 클라우드라고 부릅니다.', video=196, video_link='https://example.com/video/196', time=[770.816, 772.557], source='ID: 10347', relevance_score=1.0, faithfulness_score=1.0), DialoguesStrip(content='우리의 대단한 파트너 CoreWeave는 공공장소에서 공개되고 있습니다.', video=196, video_link='https://example.com/video/196', time=[772.577, 775.579], source='ID: 10348', relevance_score=0.6, faithfulness_score=0.8), DialoguesStrip(content='정말 자랑스럽습니다.', video=196, video_link='https://example.com/video/196', time=[775.619, 776.479], source='ID: 10349', relevance_score=0.3, faithfulness_score=0.3), DialoguesStrip(content='GPU 클라우드는 자기들만의 요구를 가지고 있습니다.', video=196, video_link='https://example.com/video/196', time=[776.499, 777.68], source='ID: 10350', relevance_score=1.0, faithfulness_score=0.9)] query_relevance=0.95
#품질 필터링
        #print(f"video: {extracted_data.video}, video_link: {extracted_data.video_link}")
        # 품질 필터링
        if extracted_data.query_relevance < 0.8:
            print(f"문서 관련성이 낮음: {extracted_data.query_relevance}")
            return {"extracted_info": []}
        
        print("품질 필터링")
        # 고품질 정보만 필터링
        high_quality_strips = [
            strip for strip in extracted_data.strips
            if strip.relevance_score > 0.7 and strip.faithfulness_score > 0.7
        ]
        
        print(f"고품질 정보 {len(high_quality_strips)}개 추출됨")
        return {"extracted_info": high_quality_strips}
    
    except Exception as e:
        print(f"문서 처리 중 오류 발생: {e}")
        return {"extracted_info": []}
    


# 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# LangChain의 EmbeddingsRedundantFilter 초기화
redundant_filter = EmbeddingsRedundantFilter(
    embeddings=embeddings,
    similarity_threshold=0.85,  # 85% 이상 유사하면 중복으로 판단
)

def remove_semantic_duplicates(info_list: List[DialoguesStrip], 
                              similarity_threshold: float = 0.85) -> List[DialoguesStrip]:
    """LangChain EmbeddingsRedundantFilter를 사용한 의미적 중복 제거"""
    if not info_list:
        return []
    
    if len(info_list) == 1:
        return info_list
    
    try:
        # DialoguesStrip을 Document로 변환
        documents = []
        for i, info in enumerate(info_list):
            doc = Document(
                page_content=info.content,
                metadata={
                    "index": i,
                    "source": info.source,
                    "relevance_score": info.relevance_score,
                    "faithfulness_score": info.faithfulness_score
                }
            )
            documents.append(doc)
        
        # 임계값이 기본값과 다르면 새로운 필터 생성
        if similarity_threshold != 0.85:
            filter_instance = EmbeddingsRedundantFilter(
                embeddings=embeddings,
                similarity_threshold=similarity_threshold,
            )
        else:
            filter_instance = redundant_filter
        
        # 중복 제거 수행
        unique_documents = filter_instance.transform_documents(documents)
        
        # Document를 다시 DialoguesStrip으로 변환
        unique_info = []
        for doc in unique_documents:
            original_index = doc.metadata["index"]
            unique_info.append(info_list[original_index])
        
        # 품질 점수 순으로 정렬
        unique_info.sort(
            key=lambda x: (x.relevance_score + x.faithfulness_score) / 2, 
            reverse=True
        )
        
        print(f"EmbeddingsRedundantFilter 중복 제거: {len(info_list)} → {len(unique_info)}개")
        return unique_info
        
    except Exception as e:
        print(f"EmbeddingsRedundantFilter 처리 중 오류: {e}")
        return info_list


def aggregate_results(state: SynopsisDialoguesRagState) -> SynopsisDialoguesRagState:
    """병렬 처리된 결과를 집계하는 함수"""
    print(f"---synopsis dialogues 관련 총 {len(state['extracted_info'])}개의 정보 조각 집계---")
    
    if not state["extracted_info"]:
        return {
            "num_generations": state.get("num_generations", 0) + 1,
            "processed_info": []
        }
    
    # 1. LangChain EmbeddingsRedundantFilter를 사용한 의미적 중복 제거
    unique_info = remove_semantic_duplicates(
        state["extracted_info"], 
        similarity_threshold=0.85  # 85% 이상 유사하면 중복으로 판단
    )
    
    # 2. 품질 점수에 따른 정렬
    sorted_info = sorted(
        unique_info, 
        key=lambda x: (x.relevance_score + x.faithfulness_score) / 2, 
        reverse=True
    )
    
    # 3. 상위 N개만 선택 
    top_info = sorted_info[:SEARCH_TOP_K]  # 상위 10개만 유지
    
    print(f"최종 결과: {len(state['extracted_info'])} → {len(unique_info)}  → {len(top_info)}개")
    
    # 중요: extracted_info 키를 반환하지 않음 (reducer 중복 방지)
    return {
        "num_generations": state.get("num_generations", 0) + 1,
        "processed_info": top_info  # 후처리된 결과
    }


def rewrite_query(state: SynopsisDialoguesRagState) -> SynopsisDialoguesRagState:
    print("---synopsis dialogues 쿼리 재작성---")

    rewrite_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 video 분석 전문가입니다. 주어진 원래 질문과 추출된 정보를 바탕으로, 더 관련성 있고 충실한 정보를 찾기 위해 검색 쿼리를 개선해주세요.

         주어진 원래 질문과 추출된 정보를 바탕으로, 더 관련성 있고 충실한 정보를 찾기 위해 검색 쿼리를 개선해주세요.
         검색된 내용에 실행 가능한 링크가 있으면 반드시 추가해주세요.

        다음 사항을 고려하여 검색 쿼리를 개선하세요:
        1. 원래 질문의 핵심 요소
        2. 추출된 정보의 관련성 점수
        3. 추출된 정보의 충실성 점수
        4. 부족한 정보나 더 자세히 알아야 할 부분

        개선된 검색 쿼리 작성 단계:
        1. 2-3개의 검색 쿼리를 제안하세요.
        2. 각 쿼리는 구체적이고 간결해야 합니다(5-10 단어 사이).
        3. 각 쿼리 뒤에는 해당 쿼리를 제안한 이유를 간단히 설명하세요.

        출력 형식:
        1. [개선된 검색 쿼리 1]
        - 이유: [이 쿼리를 제안한 이유 설명]
        2. [개선된 검색 쿼리 2석
        - 이유: [이 쿼리를 제안한 이유 설명]
        3. [개선된 검색 쿼리 3]
        - 이유: [이 쿼리를 제안한 이유 설명]

        마지막으로, 제안된 쿼리 중 가장 효과적일 것 같은 쿼리를 선택하고 그 이유를 설명하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<원래 질문>\n{question}\n</원래 질문>\n\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 개선된 검색 쿼리를 작성해주세요.")
    ])

    processed_info_str = "\n".join([strip.content for strip in state["processed_info"]])
    
    rewrite_llm = llm.with_structured_output(RefinedQuestion)

    response = rewrite_llm.invoke(rewrite_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"쿼리를 재작성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))
    
    return {"rewritten_query": response.question_refined}


def generate_node_answer(state: SynopsisDialoguesRagState) -> SynopsisDialoguesRagState:
    print("---synopsis dialogues 답변 생성---")

    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 video 분석 전문가입니다. 주어진 질문과 추출된 정보를 바탕으로 답변을 생성해주세요. 
        답변은 마크다운 형식으로 작성하며, 각 정보의 출처를 명확히 표시해야 합니다.석
        답변 구조:
        1. 추출된 대화 내용을 제공
        2. 각 대화별로 영상 링크 제공
        3. 결론 및 요약
         - 요약 정보에는 주요 키워드를 3~5개 추가하고 중복인 경우, 개수를 '(3)' 와 같이 표시
        각 섹션에서 사용된 정보의 출처를 괄호 안에 명시하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 최종 답변을 작성해주세요.")
    ])


    for strip in state["processed_info"]:
        print(f"strip: {strip}")
        video = strip.video
        url_prefix = "http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3"
        request_url=f"http://metavision.k8s.lightningdb/api/metavision-ego-view-ai/api/v1/video/name?video_id={video}"
        response = requests.get(request_url)
        response.raise_for_status()
        video_name = response.text.replace('"', '')
        video_link = f"{url_prefix}/original_videos/{video_name}"
        print(f"converted link: {video_link}")
        strip.video_link = video_link
        
    print(f"----> video_link: {video_link}")
    processed_info_str = "\n---\n".join([f"**대화내용**: {strip.content}\n**영상 링크**: {strip.video_link}\n**시간**: {strip.time}\n**출처**: {strip.source}\n**관련성**: {strip.relevance_score}\n**충실성**: {strip.faithfulness_score}" for strip in state["processed_info"]])
    print(f"----> processed_info_str: {processed_info_str}")
    #llm = llm_with_tools
    node_answer = llm.invoke(answer_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))

    return {"node_answer": node_answer.content}

def should_continue(state: SynopsisDialoguesRagState) -> Literal["계속", "종료"]:
    if state["num_generations"] >= 2:
        return "종료"
    if len(state["processed_info"]) > 0:
        return "종료"
    return "계속"

In [170]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# 그래프 생성
workflow = StateGraph(SynopsisDialoguesRagState)

# 노드 추가
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("evaluate_single_document", evaluate_single_document)
workflow.add_node("aggregate_results", aggregate_results)
workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("generate_answer", generate_node_answer)  

workflow.add_edge(START, "retrieve")

# 조건부 엣지로 병렬 처리 설정
workflow.add_conditional_edges(
    "retrieve",  # 문서 검색 완료
    create_evaluation_workers,  # 조건부 엣지 함수
    ["evaluate_single_document"]  # 병렬 워커 노드
)

workflow.add_edge("evaluate_single_document", "aggregate_results")
workflow.add_conditional_edges(
    "aggregate_results",
    should_continue,
    {
        "계속": "rewrite_query",
        "종료": "generate_answer"
    }
)
workflow.add_edge("rewrite_query", "retrieve")
workflow.add_edge("generate_answer", END)

# 그래프 컴파일
synopsis_dialogues_agent = workflow.compile()

# 그래프 시각화
# display(Image(labor_lawsynopsis_dialogues_agent_agent.get_graph().draw_mermaid_png()))
mermaid_code = synopsis_dialogues_agent.get_graph(xray=True).draw_mermaid()
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))
 
 

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retrieve(retrieve)
	evaluate_single_document(evaluate_single_document)
	aggregate_results(aggregate_results)
	rewrite_query(rewrite_query)
	generate_answer(generate_answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> retrieve;
	aggregate_results -. &nbsp;종료&nbsp; .-> generate_answer;
	aggregate_results -. &nbsp;계속&nbsp; .-> rewrite_query;
	evaluate_single_document --> aggregate_results;
	retrieve -.-> evaluate_single_document;
	rewrite_query --> retrieve;
	generate_answer --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [171]:
inputs = {"question": "GPU에 대해 얘기하고 있는 부분 찾아서 알려줘"}

step_no = 0
for output in synopsis_dialogues_agent.stream(inputs):
    step_no+=1
    for key, value in output.items():
        # 노드 출력
        pprint(f"================ Step {step_no} ==================")
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)

    print("\n----------------------------------------------------------\n")

---synopsis dialogues 문서 검색---
db_synopsis_dialogues_retriever.invoke 호출 결과: [Document(id='25cfcb76-fc33-499d-83f2-03f2ac107315', metadata={'source': 'data/synopsis_dialogues.txt'}, page_content='ID: 9810 | Video: 202 | Speaker: 1156 | Time: {214.085,216.166} | Text: 한 번 해보겠습니다.\nID: 9811 | Video: 202 | Speaker: 1156 | Time: {218.832,233.508} | Text:  블랙웰은 블랙웰 칩 중 한 개의 GPU입니다 우리는 그 한 개의 칩을 GPU라고 부릅니다 그게 틀렸습니다 그 이유는 NVLink 노맥레이처 등을 고장낸 것입니다 블랙웰을 고치기 전에\nID: 9812 | Video: 202 | Speaker: 1156 | Time: {237.412,240.094} | Text:  NVLink-144는 144개의 GPU에 연결되어 있습니다.\nID: 9813 | Video: 202 | Speaker: 1156 | Time: {240.314,242.235} | Text: 각 GPU는 GPU 다이이기 때문에 어떤 패키지에서 연결이 가능합니다.'), Document(id='3bb2c724-3c0a-4f31-84ad-ee4d9ab80ccb', metadata={'source': 'data/synopsis_dialogues.txt'}, page_content='ID: 6932 | Video: 145 | Speaker: 925 | Time: {766.493,770.516} | Text:  그들은 GPUs를 운영합니다.\nID: 6933 | Video: 145 | Speaker: 925 | Time: {770.816,772.557} | Text: GPU 클라우드라고 부릅니다.\nID: 6934 | Video: 145 |

KeyboardInterrupt: 

In [41]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['node_answer']))

```markdown
# 1. 추출된 대화 내용

- "그들은 GPUs를 운영합니다."  
- "GPU 클라우드라고 부릅니다."  
- "GPU 클라우드는 자기들만의 요구를 가지고 있습니다."  
- "블랙웰은 블랙웰 칩 중 한 개의 GPU입니다 우리는 그 한 개의 칩을 GPU라고 부릅니다 그게 틀렸습니다 그 이유는 NVLink 노맥레이처 등을 고장낸 것입니다 블랙웰을 고치기 전에"  
- "NVLink-144는 144개의 GPU에 연결되어 있습니다."  
- "각 GPU는 GPU 다이이기 때문에 어떤 패키지에서 연결이 가능합니다."  

(출처: 문서 내용)

# 2. 각 대화별 영상 링크

| 대화 내용 | 영상 링크 | 시간 (초) |
|------------|------------|------------|
| 그들은 GPUs를 운영합니다. | [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/RT_GTC2025_2.mp4) | 766.493 ~ 770.516 |
| GPU 클라우드라고 부릅니다. | [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/RT_GTC2025_2.mp4) | 770.816 ~ 772.557 |
| GPU 클라우드는 자기들만의 요구를 가지고 있습니다. | [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/RT_GTC2025_2.mp4) | 776.499 ~ 777.68 |
| 블랙웰은 블랙웰 칩 중 한 개의 GPU입니다 ... | [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/RT_GTC2025_3.mp4) | 218.832 ~ 233.508 |
| NVLink-144는 144개의 GPU에 연결되어 있습니다. | [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/RT_GTC2025_3.mp4) | 237.412 ~ 240.094 |
| 각 GPU는 GPU 다이이기 때문에 어떤 패키지에서 연결이 가능합니다. | [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/RT_GTC2025_3.mp4) | 240.314 ~ 242.235 |

(출처: 문서 내용)

# 3. 결론 및 요약

GPU에 관한 대화는 주로 GPU 클라우드 운영과 GPU 아키텍처에 대한 설명으로 구성되어 있습니다.  
특히, GPU 클라우드가 특정 요구사항을 가지고 있음을 언급하며, 블랙웰 칩과 NVLink 기술을 통해 GPU 간 연결 및 통신 구조에 대해 상세히 설명하고 있습니다.  
이 내용은 GPU의 하드웨어적 구성과 클라우드 환경에서의 운영 방식을 이해하는 데 중요한 정보를 제공합니다.

**주요 키워드**: GPU, GPU 클라우드, 블랙웰, NVLink, GPU 다이

(출처: 문서 내용)
```

`Video frame(장면) 검색 RAG Agent`

In [42]:
# 비디오제목: car_wash_skilled.mp4, 비디오링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4, 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4/grid, 생성시간: 2025-08-04 13:09:38

class Document(BaseModel):
    title: str
    video_link: str
    thumbnail: str
    created_time: str
    
class VideoSearchRagState(CorrectiveRagState):
    rewritten_query: str   # 재작성한 질문 
    video_link: str
    processed_info: List[InformationStrip]  # 후처리된 결과용 
    documents: List[Document]
    node_answer: Optional[str]

In [207]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.types import Send

def retrieve_documents(state: VideoSearchRagState) -> VideoSearchRagState:
    print("---어떤 장면에 대한 설명에 해당하는 영상 검색---")
    query = state.get("rewritten_query", state["question"])
    docs = metavision_search.invoke(query)
    docs = docs.replace(', ', '\n')
    print(f"docs: \n{docs}")
    return {"documents": docs, "node_answer": docs.replace('- ', '\n- ').replace('- 파일명', '# 파일명')}



In [208]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# 그래프 생성
workflow = StateGraph(VideoSearchRagState)

# 노드 추가
workflow.add_node("retrieve", retrieve_documents)

workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", END)

# 그래프 컴파일
video_search_agent = workflow.compile()

# 그래프 시각화
# display(Image(housing_law_agent.get_graph().draw_mermaid_png()))

In [209]:
from langchain_core.runnables.graph import CurveStyle
from IPython.display import Markdown, display

mermaid_code = video_search_agent.get_graph().draw_mermaid()
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retrieve(retrieve)
	__end__([<p>__end__</p>]):::last
	__start__ --> retrieve;
	retrieve --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [210]:
inputs = {"question": "분홍색 자켓을 입고 있는 여성이 있는 동영상을 찾아줘"}
res = video_search_agent.stream(inputs)
result = []
for output in res:
    for key, value in output.items():
        # 노드 출력
        #pprint(f"Node '{key}':")
        #pprint(f"Value: {value}", indent=2, width=80, depth=None)
        result.append(f"{key}:\n{value} \n\n")
    #print("\n----------------------------------------------------------\n")
print(result)

---어떤 장면에 대한 설명에 해당하는 영상 검색---
last page reached: 1
docs: 
- 파일명: car_wash_skilled.mp4
- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4)
- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4/grid
- 생성시간: 2025-08-04 13:09:38

- 파일명: car_wash_normal_1.mp4
- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4)
- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4/grid
- 생성시간: 2025-08-04 13:03:17

- 파일명: car_wash_normal_2.mp4
- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_2.mp4)
- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_2.mp4/grid
- 생성시간: 2025-08-04 13:32:21

- 파일명: 세차2_숙련작업자.mp4
- 비

In [211]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['node_answer']))


# 파일명: car_wash_skilled.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4/grid

- 생성시간: 2025-08-04 13:09:38


# 파일명: car_wash_normal_1.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4/grid

- 생성시간: 2025-08-04 13:03:17


# 파일명: car_wash_normal_2.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_2.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_2.mp4/grid

- 생성시간: 2025-08-04 13:32:21


# 파일명: 세차2_숙련작업자.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차2_숙련작업자.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차2_숙련작업자.mp4/grid

- 생성시간: 2025-08-01 13:29:51


# 파일명: 세차3_작업자1.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자1.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자1.mp4/grid

- 생성시간: 2025-08-01 13:50:50


# 파일명: 세차3_작업자2.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자2.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자2.mp4/grid

- 생성시간: 2025-08-01 13:56:36


# 파일명: Apple신제품발표회.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회.mp4/grid

- 생성시간: 2025-09-02 09:16:19


# 파일명: Apple신제품발표회_2.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_2.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_2.mp4/grid

- 생성시간: 2025-09-02 09:19:29


# 파일명: Apple신제품발표회_3.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_3.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_3.mp4/grid

- 생성시간: 2025-09-02 09:27:52


# 파일명: Apple신제품발표회_4.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_4.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_4.mp4/grid

- 생성시간: 2025-09-02 09:32:41


# 파일명: Apple신제품발표회_5.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_5.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_5.mp4/grid

- 생성시간: 2025-09-02 09:40:33


# 파일명: Apple신제품발표회_6.mp4

- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_6.mp4)

- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_6.mp4/grid

- 생성시간: 2025-09-02 10:25:28



In [48]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['documents'].replace('- ', '\n- ').replace('- 비디오 링크', '\t - 비디오 링크').replace('- 썸네일', '\t - 썸네일').replace('- 생성시간', '\t - 생성시간')))


- 파일명: car_wash_skilled.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4/grid

	 - 생성시간: 2025-08-04 13:09:38


- 파일명: car_wash_normal_1.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4/grid

	 - 생성시간: 2025-08-04 13:03:17


- 파일명: car_wash_normal_2.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_2.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_2.mp4/grid

	 - 생성시간: 2025-08-04 13:32:21


- 파일명: 세차2_숙련작업자.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차2_숙련작업자.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차2_숙련작업자.mp4/grid

	 - 생성시간: 2025-08-01 13:29:51


- 파일명: 세차3_작업자1.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자1.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자1.mp4/grid

	 - 생성시간: 2025-08-01 13:50:50


- 파일명: 세차3_작업자2.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자2.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자2.mp4/grid

	 - 생성시간: 2025-08-01 13:56:36


- 파일명: Apple신제품발표회.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회.mp4/grid

	 - 생성시간: 2025-09-02 09:16:19


- 파일명: Apple신제품발표회_2.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_2.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_2.mp4/grid

	 - 생성시간: 2025-09-02 09:19:29


- 파일명: Apple신제품발표회_3.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_3.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_3.mp4/grid

	 - 생성시간: 2025-09-02 09:27:52


- 파일명: Apple신제품발표회_4.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_4.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_4.mp4/grid

	 - 생성시간: 2025-09-02 09:32:41


- 파일명: Apple신제품발표회_5.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_5.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_5.mp4/grid

	 - 생성시간: 2025-09-02 09:40:33


- 파일명: Apple신제품발표회_6.mp4

	 - 비디오 링크: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_6.mp4

	 - 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_6.mp4/grid

	 - 생성시간: 2025-09-02 10:25:28



### 4-2. 질문 라우팅 
- 사용자의 질문을 분석하여 적절한 에이전트를 선택 (Adaptive RAG 적용)

In [212]:
from typing import Annotated
from operator import add

# 딕셔서리 병합을 위한 사용자 정의 리듀서 
def merge_agent_responses(left: dict, right: dict) -> dict:
    """에이전트 응답들을 병합하는 리듀서"""
    if left is None:
        left = {}
    if right is None:
        right = {}
    
    # 딕셔너리 병합
    merged = left.copy()
    merged.update(right)
    return merged

# 상태 정의
class ResearchAgentState(TypedDict):
    question: str
    answers: Annotated[List[str], add]  # 각 에이전트의 답변들
    agent_responses: Annotated[dict, merge_agent_responses]  # 병렬 처리 지원 리듀서
    final_answer: str
    datasources: List[str]
    evaluation_report: Optional[dict]
    iteration_count: Optional[int]
    user_decision: Literal["continue", "stop"]  # 사용자 결정 (계속/종료)
    user_feedback: Optional[str]  # 사용자 피드백

In [213]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

# 라우팅 모델 정의
class ToolSelector(BaseModel):
    """Routes the user question to the most appropriate tool."""
    tool: Literal["search_video", "search_event", "search_dialogue"] = Field(
        description="Select one of the tools, based on the user's question.",
    )

class ToolSelectors(BaseModel):
    """Select the appropriate tools that are suitable for the user question."""
    tools: List[ToolSelector] = Field(
        description="Select one or more tools, based on the user's question.",
    )

# 라우팅 시스템
structured_llm_tool_selector = llm.with_structured_output(ToolSelectors)

# 라우팅 시스템 프롬프트
system = dedent("""You are an AI assistant specializing in routing user questions to the appropriate tools.
Use the following guidelines to select the most suitable tool(s):

## 비디오 검색 도구
- *semantic search*: 비디오 속 한 장면에 대한 묘사로 질문
- *event synopsis*: 비디오 전체 상황에 대한 설명으로 질문
- *dialogues*: 비디오 속에서 말하거나, 대화 또는 이야기를 하고 있는 내용에 대한 질문


Always choose all of the appropriate tools based on the user's question.
If a question is about a law but doesn't seem to be asking about specific legal provisions, include both the relevant law search tool and the search_web tool.
""")

route_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder("user_feedback"),
    ("human", "{question}"),
])

question_tool_router = route_prompt | structured_llm_tool_selector

    
# 테스트 실행
print(question_tool_router.invoke({"question": "분홍색 자켓을 입은 여자가 나오는 비디오 찾아줘", "user_feedback": []}))
print(question_tool_router.invoke({"question": "경찰이 단속하는 내용이 있는 비디오 찾아서 어떤 사건들이 있었는지 정리해줘", "user_feedback": []}))
print(question_tool_router.invoke({"question": "NVIDIA 관련 이야기하는 비디오 찾아서 주요 내용을 요약해줘", "user_feedback": []}))
print(question_tool_router.invoke({"question": "GPU에 대해 대화하고 있는 비디오 찾아서 주요 내용을 요약해줘", "user_feedback": []}))
print(question_tool_router.invoke({"question": "요리 방법에 대해 설명하고 있는 비디오 찾아서 그 방법을 설명해줘", "user_feedback": []}))
print(question_tool_router.invoke({"question": "목공에 대해 설명하고 있는 비디오 찾아서 어떤 환경에서 작업하는지 알려줘", "user_feedback": []}))

tools=[ToolSelector(tool='search_video')]
tools=[ToolSelector(tool='search_event')]
tools=[ToolSelector(tool='search_video')]
tools=[ToolSelector(tool='search_dialogue')]
tools=[ToolSelector(tool='search_event')]
tools=[ToolSelector(tool='search_event')]


In [214]:
def analyze_question_tool_search(state: ResearchAgentState):
    """질문을 분석하여 적절한 도구들을 선택"""
    question = state["question"]
    print(f"도구 선택을 위해 질문 분석 중: {question}")
    
    # 사용자 피드백이 있는 경우, 도구 선택에 반영
    user_feedback = state.get("user_feedback", None)

    result = question_tool_router.invoke({
        "question": question, 
        "user_feedback": [("system", f"***You SHOULD consider the following user feedback when selecting tools:***\n<User feedback>{user_feedback}</User feedback>")] if user_feedback else []
    })

    datasources = [tool.tool for tool in result.tools]
    print(f"선택된 도구들: {datasources}")
    return {"datasources": datasources, "agent_responses": {}}

def route_datasources_tool_search(state: ResearchAgentState) -> List[str]:
    """선택된 데이터 소스들로 라우팅"""
    datasources = set(state['datasources'])
    valid_sources = {"search_video", "search_event", "search_dialogue"}
    
    if datasources.issubset(valid_sources):
        return list(datasources)
    
    return ["search_event"]  # 유효한 도구 호출이 없는 경우, 기본적으로 search_event 사용

In [215]:
# 최종 답변 생성 프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert assistant specializing in synthesizing and integrating multiple agent responses to provide comprehensive, accurate answers.

## Core Guidelines

### 1. Information Integration Strategy
- 당신은 비디오 분석 전문가로 필요한 비디오 데이터를 요청받은 설명을 기반으로 가장 적합한 비디오를 검색하여 링크와 설명을 제공한다.
- 검색한 결과는 최대한 많은 정보를 포함하여 제공한다.
- 출처는 검색한 파일명으로 표시한다.


### 2. Special Instructions
- **Never discard valuable information** from any agent unless it's clearly incorrect
- **Always attempt to find connections** between different pieces of information
- **Include quantitative data** (dates, amounts, percentages) when available

Only say "제공된 정보로는 충분한 답변을 할 수 없습니다" if NO agents provided relevant information.
"""
    ),
    MessagesPlaceholder("user_feedback"),
    ("human", """Synthesize the following agent responses to provide a comprehensive answer to the user's question.

<Question>
{question}
</Question>

<Agent Responses>
{agent_responses}
</Agent Responses>

Create a comprehensive, well-structured answer that maximizes the value of all provided information."""),
])

# 에이전트 응답 포맷터
def format_agent_responses(agent_responses_dict):
    """에이전트 응답들을 프롬프트에 적합한 형태로 포맷팅"""
    if not agent_responses_dict:
        return "에이전트 응답이 없습니다."
    
    formatted_responses = []
    agent_labels = {
        'search_video': '🎥 비디오 Semantic Search 에이전트',
        'search_event': '🎭 SynapsEgo 비디오 분석 및 검색 에이전트',
        'search_dialogue': '💬 대화 검색 에이전트'
    }
    
    for agent_type, response in agent_responses_dict.items():
        if response and response.strip():
            label = agent_labels.get(agent_type, f'📋 {agent_type} 에이전트')
            formatted_responses.append(f"{label}:\n{response.strip()}\n")
    
    return "\n" + "="*50 + "\n".join(formatted_responses) + "="*50

In [216]:
def synopsis_event_rag_node(state: ResearchAgentState) -> ResearchAgentState:
    """비디오 이벤트 검색 에이전트"""
    print("--- 비디오 이벤트 검색 에이전트 시작 ---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    answer = synopsis_event_agent.invoke({
        "question": question, 
        "user_feedback": [("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    response_text = answer.get("node_answer", "답변을 생성할 수 없습니다.")
    
    # 에이전트별 응답 저장
    current_responses = state.get("agent_responses", {}).copy()
    current_responses["search_response"] = response_text
    
    return {
        "answers": [response_text],
        "agent_responses": current_responses
    }

def synopsis_dialogues_rag_node(state: ResearchAgentState) -> ResearchAgentState:
    """비디오 대화 검색 에이전트"""
    print("--- 비디오 대화 검색 에이전트 시작 ---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    answer = synopsis_dialogues_agent.invoke({
        "question": question, 
        "user_feedback": [("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    response_text = answer.get("node_answer", "답변을 생성할 수 없습니다.")
    current_responses = state.get("agent_responses", {}).copy()
    current_responses["search_response"] = response_text
    
    return {
        "answers": [response_text],
        "agent_responses": current_responses
    }

def video_search_rag_node(state: ResearchAgentState) -> ResearchAgentState:
    """비디오 Semantic Search 에이전트"""
    print("--- 비디오 Semantic Search 에이전트 시작 ---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    answer = video_search_agent.invoke({
        "question": question, 
        "user_feedback": [("user", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    response_text = answer.get("node_answer", "답변을 생성할 수 없습니다.")

    current_responses = state.get("agent_responses", {}).copy()
    current_responses["search_response"] = response_text

    return {
        "answers": [response_text],
        "agent_responses": current_responses
    }


def answer_final(state: ResearchAgentState) -> dict:
    """최종 답변 생성 함수"""
    print("---최종 답변 생성---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)
    # 에이전트 응답들을 가져옴
    agent_responses = state.get("agent_responses", {})
    
    # 에이전트 응답이 있는지 확인
    if not agent_responses:
        return {
            "final_answer": "에이전트로부터 답변을 받지 못했습니다.",
            "question": question
        }
    
    # 에이전트 응답 포맷팅
    formatted_responses = format_agent_responses(agent_responses)
    
    print(f"통합할 에이전트 응답 수: {len(agent_responses)}")
    for agent_type in agent_responses.keys():
        print(f"  - {agent_type}")
    
    # RAG 체인으로 최종 답변 생성
    rag_chain = rag_prompt | llm | StrOutputParser()

    generation = rag_chain.invoke({
        "question": question,
        "agent_responses": formatted_responses,
        "user_feedback": [("system", f"최종 답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    print(f"최종 답변 생성 완료: {generation[:100]}...")  # 처음 100자만 출력

    return {
        "final_answer": generation,
        "question": question
    }

# ===== LLM Fallback =====
fallback_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant helping with various topics. Follow these guidelines:
1. Provide accurate and helpful information to the best of your ability.
2. Express uncertainty when unsure; avoid speculation.
3. Keep answers concise yet informative.
4. Respond ethically and constructively."""),
    MessagesPlaceholder("user_feedback"),
    ("human", "{question}"),
])

def llm_fallback(state: ResearchAgentState) -> dict:
    """LLM 일반 응답"""
    print("---Fallback 답변---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    llm_chain = fallback_prompt | llm | StrOutputParser()
    generation = llm_chain.invoke({
        "question": question,
        "user_feedback": [("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    print(f"Fallback 답변 생성 완료: {generation[:100]}...")  # 처음 100자만 출력
    return {
        "final_answer": generation,
        "question": question
    }

In [217]:
# 노드 정의를 딕셔너리로 관리
nodes = {
    "analyze_question": analyze_question_tool_search,
    "search_event": synopsis_event_rag_node,
    "search_dialogue": synopsis_dialogues_rag_node,
    "search_video": video_search_rag_node,
    "generate_answer": answer_final
}

# 그래프 생성
search_builder = StateGraph(ResearchAgentState)

# 노드 추가
for node_name, node_func in nodes.items():
    search_builder.add_node(node_name, node_func)

# 엣지 추가
search_builder.add_edge(START, "analyze_question")

# 조건부 엣지 (병렬 처리)
search_builder.add_conditional_edges(
    "analyze_question",
    route_datasources_tool_search,
    ["search_video", "search_event", "search_dialogue"]
)

# 검색 노드들을 generate_answer에 연결
for node in ["search_event", "search_dialogue", "search_video"]:
    search_builder.add_edge(node, "generate_answer")
#search_builder.add_edge("search_video", END)
search_builder.add_edge("generate_answer", END)

# 그래프 컴파일
rag_search_graph = search_builder.compile()

# 그래프 시각화
# display(Image(rag_search_graph.get_graph().draw_mermaid_png()))

In [218]:
from langchain_core.runnables.graph import CurveStyle
from IPython.display import Markdown, display

mermaid_code = rag_search_graph.get_graph().draw_mermaid()
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	analyze_question(analyze_question)
	search_event(search_event)
	search_dialogue(search_dialogue)
	search_video(search_video)
	generate_answer(generate_answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> analyze_question;
	analyze_question -.-> search_dialogue;
	analyze_question -.-> search_event;
	analyze_question -.-> search_video;
	search_dialogue --> generate_answer;
	search_event --> generate_answer;
	search_video --> generate_answer;
	generate_answer --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [ ]:
# 그래프 시각화
# display(Image(rag_search_graph.get_graph(xray=True).draw_mermaid_png()))

In [147]:
from langchain_core.runnables.graph import CurveStyle
from IPython.display import Markdown, display

mermaid_code = rag_search_graph.get_graph(xray=True).draw_mermaid()
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	analyze_question(analyze_question)
	generate_answer(generate_answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> analyze_question;
	analyze_question -.-> search_dialogue___start__;
	analyze_question -.-> search_event___start__;
	analyze_question -.-> search_video_retrieve;
	search_dialogue_generate_answer --> generate_answer;
	search_event_generate_answer --> generate_answer;
	search_video_retrieve --> generate_answer;
	generate_answer --> __end__;
	subgraph search_event
	search_event___start__(<p>__start__</p>)
	search_event_retrieve(retrieve)
	search_event_evaluate_single_document(evaluate_single_document)
	search_event_aggregate_results(aggregate_results)
	search_event_rewrite_query(rewrite_query)
	search_event_generate_answer(generate_answer)
	search_event___start__ --> search_event_retrieve;
	search_event_aggregate_results -. &nbsp;종료&nbsp; .-> search_event_generate_answer;
	search_event_aggregate_results -. &nbsp;계속&nbsp; .-> search_event_rewrite_query;
	search_event_evaluate_single_document --> search_event_aggregate_results;
	search_event_retrieve -.-> search_event_evaluate_single_document;
	search_event_rewrite_query --> search_event_retrieve;
	end
	subgraph search_dialogue
	search_dialogue___start__(<p>__start__</p>)
	search_dialogue_retrieve(retrieve)
	search_dialogue_evaluate_single_document(evaluate_single_document)
	search_dialogue_aggregate_results(aggregate_results)
	search_dialogue_rewrite_query(rewrite_query)
	search_dialogue_generate_answer(generate_answer)
	search_dialogue___start__ --> search_dialogue_retrieve;
	search_dialogue_aggregate_results -. &nbsp;종료&nbsp; .-> search_dialogue_generate_answer;
	search_dialogue_aggregate_results -. &nbsp;계속&nbsp; .-> search_dialogue_rewrite_query;
	search_dialogue_evaluate_single_document --> search_dialogue_aggregate_results;
	search_dialogue_retrieve -.-> search_dialogue_evaluate_single_document;
	search_dialogue_rewrite_query --> search_dialogue_retrieve;
	end
	subgraph search_video
	search_video_retrieve(retrieve)
	end
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [148]:
# 1개의 도구가 선택된 경우
inputs = {
    "question": "경찰이 단속하는 내용이 있는 비디오 찾아서 어떤 사건들이 있었는지 정리해줘"
}
for output in rag_search_graph.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

도구 선택을 위해 질문 분석 중: 경찰이 단속하는 내용이 있는 비디오 찾아서 어떤 사건들이 있었는지 정리해줘
선택된 도구들: ['search_event']
"Node 'analyze_question':"
"Value: {'datasources': ['search_event'], 'agent_responses': {}}"

----------------------------------------------------------

--- 비디오 이벤트 검색 에이전트 시작 ---
---Synopsis event 검색---
---Synopsis event 관련 10개 문서에 대한 평가 병렬 워커 생성---
---Synopsis event 단일 문서 처리 중 (길이: 745...)---
---Synopsis event 단일 문서 처리 중 (길이: 736...)---
---Synopsis event 단일 문서 처리 중 (길이: 736...)---
---Synopsis event 단일 문서 처리 중 (길이: 736...)---
---Synopsis event 단일 문서 처리 중 (길이: 508...)---
---Synopsis event 단일 문서 처리 중 (길이: 508...)---
---Synopsis event 단일 문서 처리 중 (길이: 508...)---
---Synopsis event 단일 문서 처리 중 (길이: 787...)---
---Synopsis event 단일 문서 처리 중 (길이: 787...)---
---Synopsis event 단일 문서 처리 중 (길이: 787...)---
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
---Synopsis event 관련 총 10개의 정보 조각 집계---
---Synopsis event 답변 생성---
"Nod

In [149]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['final_answer']))

경찰 단속 및 사건 관련 비디오 4건을 정리해 드립니다. 각 사건의 주요 내용과 발생 일시, 장소, 관련 태그, 그리고 영상 링크를 포함하여 상세히 안내합니다.

---

1. **공원에서의 야간 경찰 체포**  
   - **내용:** 야간 공원에서 파란색 차량을 운전하던 여성이 경찰에 의해 체포되는 장면입니다. 경찰관이 운전자에게 손을 등 뒤로 놓으라고 지시하였고, 여성은 잠시 저항 후 준수하여 경찰 차량으로 안내되어 체포가 완료되었습니다. 사건은 가로등 조명 아래 주거 지역 거리에서 발생했습니다.  
   - **발생 일시:** 2025년 8월 29일 14:54:16 ~ 14:55:19  
   - **태그:** #경찰체포, #야간체포, #주거지역거리  
   - **영상 링크:** [바로가기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/Police_Arrest1/events/Police_Arrest1_event_139.mp4)

2. **빨간불 위반으로 인한 교통 정지 (중복 2건)**  
   - **내용:** 경기 성남시 분당구 판교로 인근 도시 거리에서 경찰관이 신체 카메라를 사용해 흰색 트래버스 SUV(번호판 236조 2693)를 빨간불 위반 혐의로 정지시켰습니다. 교통 카메라 영상으로 좌회전 중 빨간불 위반 사실이 확인되었고, 운전자 최민호 씨에게 15점 벌점과 7만원 벌금이 부과되었습니다. 운전자는 처음에 위반 사실을 부인했으나 경찰이 증거를 제시하며 인정하게 되었습니다.  
   - **발생 일시:** 2025년 8월 4일 20:03:05 ~ 20:06:47  
   - **태그:** #교통위반, #빨간불카메라, #운전벌점  
   - **영상 링크:** [바로가기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/GX018665/events/GX018665_event_90.mp4)

3. **경찰 사건: 물 요청 및 아동 구속 (중복 2건)**  
   - **내용:** 주거 지역에서 경찰관들이 남성과 아동을 구속하는 사건이 발생했습니다. 구체적인 사건 경위는 영상에서 확인할 수 있습니다.  
   - **발생 일시:** 2025년 9월 26일 07:19:03 ~ 07:21:03  
   - **태그:** #경찰사건, #물요청, #아동구속  
   - **영상 링크:** [바로가기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/RT_2_eXsY-F1HLzg/events/RT_2_eXsY-F1HLzg_event_275.mp4)

4. **경찰견(K-9)이 숲에서 실종 여성 발견 (중복 2건)**  
   - **내용:** 보안관의 경찰견(K-9) 부대가 주택가 인근 숲에서 실종된 여성을 찾기 위해 낮 시간 수색을 벌였습니다. 경찰견 판도라와 리나가 동네와 숲을 수색한 끝에 흰 머리와 파란 셔츠를 입은 실종 여성을 발견하는 데 성공했습니다.  
   - **발생 일시:** 2025년 8월 29일 13:32:25 ~ 13:34:39  
   - **태그:** #경찰K-9, #실종자, #K-9팀  
   - **영상 링크:** [바로가기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/videos/2ml6NyNJlmg/events/2ml6NyNJlmg_event_138.mp4)

---

### 종합 요약  
- 총 4개의 고유한 경찰 단속 및 사건 영상이 확인되었습니다.  
- 주요 사건 유형은 야간 체포, 교통 위반 단속, 아동 구속 사건, 그리고 경찰견을 활용한 실종자 수색입니다.  
- 각 사건은 2025년 8월과 9월에 발생했으며, 주로 주거 지역과 도시 거리, 숲 인근에서 진행되었습니다.  
- 교통 위반 사건에서는 벌점 15점과 벌금 7만원이 부과되는 등 구체적인 법적 조치가 이루어졌습니다.  
- 경찰견 수색 사건은 실종자 발견이라는 긍정적 결과를 포함하고 있습니다.

필요하신 경우 각 영상 링크를 통해 사건의 구체적인 상황을 직접 확인하실 수 있습니다.

In [150]:

inputs = {"question": "GPU에 대해 이야기하고 있는 비디오 찾아줘"}
for output in rag_search_graph.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

도구 선택을 위해 질문 분석 중: GPU에 대해 이야기하고 있는 비디오 찾아줘
선택된 도구들: ['search_dialogue']
"Node 'analyze_question':"
"Value: {'datasources': ['search_dialogue'], 'agent_responses': {}}"

----------------------------------------------------------

--- 비디오 대화 검색 에이전트 시작 ---
---synopsis dialogues 문서 검색---
db_synopsis_dialogues_retriever.invoke 호출 결과: [Document(id='3bb2c724-3c0a-4f31-84ad-ee4d9ab80ccb', metadata={'source': 'data/synopsis_dialogues.txt'}, page_content='ID: 6932 | Video: 145 | Speaker: 925 | Time: {766.493,770.516} | Text:  그들은 GPUs를 운영합니다.\nID: 6933 | Video: 145 | Speaker: 925 | Time: {770.816,772.557} | Text: GPU 클라우드라고 부릅니다.\nID: 6934 | Video: 145 | Speaker: 925 | Time: {772.577,775.579} | Text: 우리의 대단한 파트너 CoreWeave는 공공장소에서 공개되고 있습니다.\nID: 6935 | Video: 145 | Speaker: 925 | Time: {775.619,776.479} | Text: 정말 자랑스럽습니다.\nID: 6936 | Video: 145 | Speaker: 925 | Time: {776.499,777.68} | Text: GPU 클라우드는 자기들만의 요구를 가지고 있습니다.'), Document(id='e58f7f42-b4d7-46c7-99ca-db0983c319d7', metadata={'source

In [151]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['final_answer']))

GPU에 대해 이야기하는 비디오는 GTC2025 컨퍼런스 관련 영상에서 찾을 수 있습니다. 주요 대화 내용은 다음과 같습니다.

- "그들은 GPUs를 운영합니다."
- "GPU 클라우드라고 부릅니다."
- "GPU 클라우드는 자기들만의 요구를 가지고 있습니다."

이 대화들은 GPU의 운영과 GPU 클라우드 환경에서의 활용 및 요구사항에 대해 설명하고 있습니다. 관련 영상은 다음 링크에서 확인할 수 있으며, 각각의 대화가 나온 시간 구간도 함께 안내드립니다.

| 대화 내용                      | 영상 링크                                                                                              | 시간 (초)          |
|-----------------------------|---------------------------------------------------------------------------------------------------|------------------|
| 그들은 GPUs를 운영합니다.          | [GTC2025컨퍼런스_2.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/GTC2025컨퍼런스_2.mp4) | 766.493 ~ 770.516 |
| GPU 클라우드라고 부릅니다.         | [GTC2025컨퍼런스_2.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/GTC2025컨퍼런스_2.mp4) | 770.816 ~ 772.557 |
| GPU 클라우드는 자기들만의 요구를 가지고 있습니다. | [RT_GTC2025컨퍼런스_2.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/RT_GTC2025컨퍼런스_2.mp4) | 776.499 ~ 777.68  |

이 영상들은 GPU의 운영 방식과 클라우드 환경에서 GPU가 어떻게 활용되는지, 그리고 그에 따른 특수한 요구사항에 대해 다루고 있어 GPU 관련 내용을 이해하는 데 유용합니다.

In [219]:
inputs = {"question": "분홍색 자켓을 입고 있는 여자가 있는 비디오를 찾아줘"}

for output in rag_search_graph.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")


도구 선택을 위해 질문 분석 중: 분홍색 자켓을 입고 있는 여자가 있는 비디오를 찾아줘
선택된 도구들: ['search_video']
"Node 'analyze_question':"
"Value: {'datasources': ['search_video'], 'agent_responses': {}}"

----------------------------------------------------------

--- 비디오 Semantic Search 에이전트 시작 ---
---어떤 장면에 대한 설명에 해당하는 영상 검색---
last page reached: 1
docs: 
- 파일명: car_wash_skilled.mp4
- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4)
- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4/grid
- 생성시간: 2025-08-04 13:09:38

- 파일명: car_wash_normal_1.mp4
- 비디오 링크: [영상 보기](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4)
- 썸네일: http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4/grid
- 생성시간: 2025-08-04 13:03:17

- 파일명: car_wash_normal_2.mp4
- 비디오 링크: [영상 보기](http://metavision.k8s.ligh

In [221]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['final_answer']))

분홍색 자켓을 입고 있는 여자가 있는 비디오를 요청하셨으나, 제공된 검색 결과 내에는 분홍색 자켓을 입은 여자가 등장하는 비디오에 대한 구체적인 설명이나 태그가 포함된 파일은 확인되지 않았습니다.

검색된 비디오 목록은 주로 세차 작업 관련 영상과 Apple 신제품 발표회 영상들로 구성되어 있으며, 각각의 파일명과 생성시간, 영상 링크는 다음과 같습니다.

1. 세차 관련 영상
- car_wash_skilled.mp4 (생성시간: 2025-08-04 13:09:38)
- car_wash_normal_1.mp4 (2025-08-04 13:03:17)
- car_wash_normal_2.mp4 (2025-08-04 13:32:21)
- 세차2_숙련작업자.mp4 (2025-08-01 13:29:51)
- 세차3_작업자1.mp4 (2025-08-01 13:50:50)
- 세차3_작업자2.mp4 (2025-08-01 13:56:36)

2. Apple 신제품 발표회 영상
- Apple신제품발표회.mp4 (2025-09-02 09:16:19)
- Apple신제품발표회_2.mp4 (2025-09-02 09:19:29)
- Apple신제품발표회_3.mp4 (2025-09-02 09:27:52)
- Apple신제품발표회_4.mp4 (2025-09-02 09:32:41)
- Apple신제품발표회_5.mp4 (2025-09-02 09:40:33)
- Apple신제품발표회_6.mp4 (2025-09-02 10:25:28)

각 영상은 아래 링크에서 확인 가능합니다.

- [car_wash_skilled.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_skilled.mp4)
- [car_wash_normal_1.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_1.mp4)
- [car_wash_normal_2.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/car_wash_normal_2.mp4)
- [세차2_숙련작업자.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차2_숙련작업자.mp4)
- [세차3_작업자1.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자1.mp4)
- [세차3_작업자2.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/세차3_작업자2.mp4)
- [Apple신제품발표회.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회.mp4)
- [Apple신제품발표회_2.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_2.mp4)
- [Apple신제품발표회_3.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_3.mp4)
- [Apple신제품발표회_4.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_4.mp4)
- [Apple신제품발표회_5.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_5.mp4)
- [Apple신제품발표회_6.mp4](http://metavision.k8s.lightningdb/api/preview/skt-svc-ego-view-ai-v3/original_videos/Apple신제품발표회_6.mp4)

현재까지의 데이터로는 분홍색 자켓을 입은 여자가 등장하는 비디오를 특정할 수 없으므로, 추가적인 키워드나 상세 설명이 있으면 더 정확한 검색이 가능할 것으로 보입니다. 필요하시면 다시 요청해 주세요.

# 6. Gradio 챗봇

In [223]:
import gradio as gr
from typing import List, Tuple

# 예시 질문들
example_questions = [
    "분홍색 자켓을 입은 여자가 나오는 비디오 찾아줘",
    "경찰이 단속하는 내용이 있는 비디오 찾아서 어떤 사건들이 있었는지 정리해줘",
    "NVIDIA 관련 이야기하는 비디오 찾아서 주요 내용을 요약해줘",
    "GPU에 대해 말하고 있는 비디오 찾아서 주요 내용을 요약해줘",
    "요리 방법에 대해 설명하고 있는 비디오 찾아서 그 방법을 설명해줘",
    "목공에 대해 설명하고 있는 비디오 찾아서 어떤 환경에서 작업하는지 알려줘"
]

# checkpoint, interrupt 등 복잡한 상태 관리 없이 간단한 Q&A만 처리하는 ChatBot 클래스
class SimpleVideoRAGChatBot:
    def __init__(self, graph):
        self.graph = graph

    def chat(self, message: str, history: List[Tuple[str, str]]) -> str:
        """질문을 받아서 답변만 반환"""
        try:
            # 질문을 graph에 전달하여 답변 생성
            inputs = {"question": message}
            final_answer = None
            # graph.stream()의 마지막 결과에서 final_answer 추출
            #final_answer = self.graph.invoke(inputs)
            for output in self.graph.stream(inputs):
                #print(f"***** output: {output}")
                if "search_event" in output:
                    final_answer = output["search_event"]["answers"][0]
                    print(f"***** search_event: {final_answer}")
                    return f"✅ **답변**\n\n{final_answer}"
                elif "search_dialogue" in output:
                    final_answer = output["search_dialogue"]["answers"][0]
                    print(f"***** search_dialogue: {final_answer}")
                    return f"✅ **답변**\n\n{final_answer}"
                elif "search_video" in output:
                    print(f"***** search_video: {output}")
                    final_answer = output["search_video"]["answers"][0]
                    print(f"***** search_video: {final_answer}")
                    return f"✅ **답변**\n\n{final_answer}"
                else:
                    print(f"***** output: {output}")
            if final_answer:
                return f"✅ **답변**\n\n{final_answer}"
            else:
                return "❌ 답변을 생성할 수 없습니다."
        except Exception as e:
            return f"❌ **오류가 발생했습니다:** {str(e)}"

# Gradio 인터페이스 생성 함수
def create_gradio_interface(chatbot):
    """Gradio 인터페이스를 생성하는 함수"""
    with gr.Blocks(
        title="Vision AI 어시스턴트 (SynapsEgo)", 
        theme=gr.themes.Soft(),
        css="""
        .chat-container { max-width: 1000px; margin: 0 auto; }
        .example-btn { margin: 2px; }
        """,
        analytics_enabled=False,
    ) as demo:
        
        # 메인 채팅 인터페이스
        chatinterface = gr.ChatInterface(
            fn=chatbot.chat,
            examples=example_questions,
            title="SynapsEgo 비디오 검색 어시스턴트",
            description="비디오 제목, 주요 내용, 비디오 링크를 포함한 비디오 검색 어시스턴트입니다.",
            type="messages",
            textbox=gr.Textbox(
                placeholder="원하는 비디오에 대해 설명해주세요.",
                container=False,
                scale=7
            ),
            analytics_enabled=False
            #render_markdown=True  # markdown 형식의 응답을 지원하도록 추가
        )
    return demo

# ChatBot 생성
chatbot = SimpleVideoRAGChatBot(rag_search_graph)

# Gradio 인터페이스 생성 및 실행
demo = create_gradio_interface(chatbot)
demo.launch(debug=True, share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


도구 선택을 위해 질문 분석 중: 요리 방법에 대해 설명하고 있는 비디오 찾아서 그 방법을 설명해줘
선택된 도구들: ['search_event']
***** output: {'analyze_question': {'datasources': ['search_event'], 'agent_responses': {}}}
--- 비디오 이벤트 검색 에이전트 시작 ---
---Synopsis event 검색---
---Synopsis event 관련 10개 문서에 대한 평가 병렬 워커 생성---
---Synopsis event 단일 문서 처리 중 (길이: 846...)---
---Synopsis event 단일 문서 처리 중 (길이: 846...)---
---Synopsis event 단일 문서 처리 중 (길이: 846...)---
---Synopsis event 단일 문서 처리 중 (길이: 843...)---
---Synopsis event 단일 문서 처리 중 (길이: 843...)---
---Synopsis event 단일 문서 처리 중 (길이: 843...)---
---Synopsis event 단일 문서 처리 중 (길이: 913...)---
---Synopsis event 단일 문서 처리 중 (길이: 750...)---
---Synopsis event 단일 문서 처리 중 (길이: 750...)---
---Synopsis event 단일 문서 처리 중 (길이: 750...)---
고품질 정보 1개 추출됨
문서 관련성이 낮음: 0.2
문서 관련성이 낮음: 0.1
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
문서 관련성이 낮음: 0.1
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
고품질 정보 1개 추출됨
---Synopsis event 관련 총 7개의 정보 조각 집계---
---Synopsis event 답변 생성---
***** search_event: - 제목: 집에서 만드는 레스토랑 스타일 스테이크  
  - 내용: 집에서 레스토